In [1]:
import stanfordnlp
nlp = stanfordnlp.Pipeline()

Use device: cpu
---
Loading: tokenize
With settings: 
{'model_path': '/home/dmlab/stanfordnlp_resources/en_ewt_models/en_ewt_tokenizer.pt', 'lang': 'en', 'shorthand': 'en_ewt', 'mode': 'predict'}
---
Loading: pos
With settings: 
{'model_path': '/home/dmlab/stanfordnlp_resources/en_ewt_models/en_ewt_tagger.pt', 'pretrain_path': '/home/dmlab/stanfordnlp_resources/en_ewt_models/en_ewt.pretrain.pt', 'lang': 'en', 'shorthand': 'en_ewt', 'mode': 'predict'}
---
Loading: lemma
With settings: 
{'model_path': '/home/dmlab/stanfordnlp_resources/en_ewt_models/en_ewt_lemmatizer.pt', 'lang': 'en', 'shorthand': 'en_ewt', 'mode': 'predict'}
Building an attentional Seq2Seq model...
Using a Bi-LSTM encoder
Using soft attention for LSTM.
Finetune all embeddings.
[Running seq2seq lemmatizer with edit classifier]
---
Loading: depparse
With settings: 
{'model_path': '/home/dmlab/stanfordnlp_resources/en_ewt_models/en_ewt_parser.pt', 'pretrain_path': '/home/dmlab/stanfordnlp_resources/en_ewt_models/en_ewt.pr

In [4]:
from DependencyGraph import DependencyGraph

/pytorch/aten/src/ATen/native/TensorAdvancedIndexing.cpp:573: UserWarning: masked_fill_ received a mask with dtype torch.uint8, this behavior is now deprecated,please use a mask with dtype torch.bool instead.


In [11]:
def extract_targets(token2idx, nodes, o, one_rel):
    new_targets = set()
    indices = token2idx[o]
    for o_idx in indices:

        # O(JJ) ~ MR <- T(NN)   *The phone has a big "screen"
        if nodes[o_idx].dep == one_rel:
            new_targets.add(nodes[nodes[o_idx].governor].token)

        # O(JJ) -> MR ~ T(NN)   *The "screen" is big
        child_nodes = [nodes[i] for i in range(len(nodes)) if nodes[i].governor==o_idx]
        new_targets.update([child_node.token for child_node in child_nodes if child_node.dep == one_rel])

    # This is for expanding the opinion targets from single word to phrases (e.g., battery life)
    return list(new_targets)

sample = 'The photo quality is amazing'
doc = nlp(sample)

sentence_graph = DependencyGraph(doc.sentences[0])
extract_targets(sentence_graph.token2idx, sentence_graph.nodes, 'amazing', 'nsubj')

/pytorch/aten/src/ATen/native/TensorAdvancedIndexing.cpp:573: UserWarning: masked_fill_ received a mask with dtype torch.uint8, this behavior is now deprecated,please use a mask with dtype torch.bool instead.


['quality']

In [34]:
def next_focused_tokens(current_tokens, token2idx, nodes, dep_rel):
    focused_tokens = set()
    for current_token in current_tokens:
        indices = token2idx[current_token]
        for current_token_idx in indices:

            if nodes[current_token_idx].dep == dep_rel: focused_tokens.add(nodes[nodes[current_token_idx].governor].token)
            child_nodes = [nodes[i] for i in range(len(nodes)) if nodes[i].governor==current_token_idx]
            focused_tokens.update([child_node.token for child_node in child_nodes if child_node.dep == dep_rel])
    return focused_tokens

def extract_targets(token2idx, nodes, o, dep_rels):

    focused_tokens = [o]
    for i in range(len(dep_rels)):
        focused_tokens = next_focused_tokens(focused_tokens, token2idx, nodes, dep_rels[i])
    
    return list(focused_tokens)


sample = 'Pod is the best mp3 player'
doc = nlp(sample)

sentence_graph = DependencyGraph(doc.sentences[0])
extract_targets(sentence_graph.token2idx, sentence_graph.nodes, 'best', ['amod', 'nsubj'])

/pytorch/aten/src/ATen/native/TensorAdvancedIndexing.cpp:573: UserWarning: masked_fill_ received a mask with dtype torch.uint8, this behavior is now deprecated,please use a mask with dtype torch.bool instead.


['Pod']

In [35]:
sample = 'The photo quality is amazing'
doc = nlp(sample)
sentence_graph = DependencyGraph(doc.sentences[0])

extract_targets(sentence_graph.token2idx, sentence_graph.nodes, 'amazing', ['nsubj'])

/pytorch/aten/src/ATen/native/TensorAdvancedIndexing.cpp:573: UserWarning: masked_fill_ received a mask with dtype torch.uint8, this behavior is now deprecated,please use a mask with dtype torch.bool instead.


['quality']